## Датасет Cityscapes

Оригинальная статья: https://www.cityscapes-dataset.com/wordpress/wp-content/papercite-data/pdf/cordts2016cityscapes.pdf

### Задача
Семантическая сегментация городских улиц.

### Полигональные аннотации
- Плотная семантическая сегментация
- Сегментация экземпляра для транспортного средства и людей

### Загрузка датасета

In [5]:
!wget 

zsh:1: no matches found: https://www.cityscapes-dataset.com/md5-sum/?packageID=1


In [3]:
dataset = Cityscapes('./data/cityscapes', split='train', mode='fine', target_type='semantic')

dataset[0]

RuntimeError: Dataset not found or incomplete. Please make sure all required folders for the specified "split" and "mode" are inside the "root" directory

### Dataloader из torchvision

In [2]:
from torchvision.datasets import Cityscapes

### Классы - 30 классов

In [ ]:
##plotly гистограмма числа экземпляров

### Описание

- 50 городов
- Несколько месяцев (весна, лето, осень)
- Дневное время
- Хорошие/средние погодные условия
- Кадры, выбранные вручную
- Большое количество динамических объектов
- Различная компоновка сцены
- Разный фон

In [7]:
import matplotlib.pyplot as plt

### Пример данных

In [6]:
dataset = Cityscapes('./data/cityscapes', split='train', mode='fine', target_type='semantic')
img, smnt = dataset[0]

RuntimeError: Dataset not found or incomplete. Please make sure all required folders for the specified "split" and "mode" are inside the "root" directory

In [ ]:
plt.imshow()